# Load data

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
print('Data loading...')

ROOT_DIR = '/Users/vivianyan/Desktop/Reinforcement-Learning/Project/RL-Movie-Rec-Sys'
DATA_DIR = os.path.join(ROOT_DIR, 'ml-100k')

ratings_list = [i.strip().split("\t") for i in open(os.path.join(DATA_DIR,'mod_ratings.csv'), 'r').readlines()]
ratings_df = pd.DataFrame(ratings_list[1:], columns = ['userId', 'movieId', 'rating', 'timestamp'])
ratings_df['userId'] = ratings_df['userId'].apply(pd.to_numeric)
ratings_df['movieId'] = ratings_df['movieId'].apply(pd.to_numeric)
ratings_df['rating'] = ratings_df['rating'].astype(float)

movies_list = [i.strip().split("\t") for i in open(os.path.join(DATA_DIR,'mod_movies.csv'),encoding='latin-1').readlines()]
movies_df = pd.DataFrame(movies_list[1:], columns = ['movieId', 'title', 'genres'])
movies_df['movieId'] = movies_df['movieId'].apply(pd.to_numeric)

users_list = [i.strip().split("\t") for i in open(os.path.join(DATA_DIR,'users.csv'), 'r').readlines()]
users_df = pd.DataFrame(users_list[1:], columns=['userId', 'gender', 'age', 'occupation', 'zip'])

tags_list = [i.strip().split("\t") for i in open(os.path.join(DATA_DIR,'mod_tags.csv'), 'r').readlines()]
tags_df = pd.DataFrame(tags_list[1:], columns=['userId', 'movieId', 'tag', 'timestamp'])
tags_df['userId'] = tags_df['userId'].apply(pd.to_numeric)
tags_df['movieId'] = tags_df['movieId'].apply(pd.to_numeric)

print("Data loading complete!")
print("Data preprocessing...")

Data loading...
Data loading complete!
Data preprocessing...


# Generate evaluation data
* movies_id_to_movies={movieId: [title, genres],}
* users_dict={userId:[movieId, rating],}
* users_history_lens=[length,]

In [3]:
print(len(movies_list))
movies_id_to_movies = {movie[0]: movie[1:] for movie in movies_list[1:]}
dict(list(movies_id_to_movies.items())[:10])

9743


{'1': ['Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 '2': ['Jumanji (1995)', 'Adventure|Children|Fantasy'],
 '3': ['Grumpier Old Men (1995)', 'Comedy|Romance'],
 '4': ['Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
 '5': ['Father of the Bride Part II (1995)', 'Comedy'],
 '6': ['Heat (1995)', 'Action|Crime|Thriller'],
 '7': ['Sabrina (1995)', 'Comedy|Romance'],
 '8': ['Tom and Huck (1995)', 'Adventure|Children'],
 '9': ['Sudden Death (1995)', 'Action'],
 '10': ['GoldenEye (1995)', 'Action|Adventure|Thriller']}

In [4]:
users_dict = {user: [] for user in set(ratings_df["userId"])}
filtered_users_dict = users_dict.copy()

# sort the dataset by time
ratings_df = ratings_df.sort_values(by='timestamp', ascending=True)

for idx, row in ratings_df.iterrows():
    users_dict[row['userId']].append((row['movieId'], row['rating']))
    if row['rating'] >= 4.0:
        filtered_users_dict[row['userId']].append((row['movieId'], row['rating']))

print(f'User dict (10 examples of UserId=1): {users_dict[1][:10]}')

users_history_lens = [len(filtered_users_dict[u]) for u in set(ratings_df["userId"])]
print(f'Length of user dict history: {len(users_history_lens)}')

User dict (10 examples of UserId=1): [(1210, 5.0), (1210, 5.0), (804, 4.0), (804, 4.0), (2018, 5.0), (2018, 5.0), (2826, 4.0), (2826, 4.0), (2628, 4.0), (2628, 4.0)]
Length of user dict history: 610


In [5]:
# np.save(os.path.join(os.getcwd(), "data/user_dict.npy"), users_dict)
# np.save(os.path.join(os.getcwd(), "data/users_histroy_len.npy"), users_history_lens)

# Train evaluation split

In [6]:
users_num = max(ratings_df['userId']) + 1
movies_num = len(set((ratings_df['movieId'])))
print(f'# users: {users_num}, # movies: {movies_num}')

# users: 611, # movies: 9724


In [7]:
train_users_num = int(users_num * 0.8)
train_movies_num = movies_num
print(f'Training set: \n# users: {train_users_num}, # movies: {train_movies_num}')

train_users_dict = {k: users_dict[k] for k in range(1, train_users_num+1)}
train_users_history_lens = users_history_lens[:train_users_num]
print(f'Length of users dict: {len(train_users_dict)}, Length of users history lens: {len(train_users_history_lens)}')

Training set: 
# users: 488, # movies: 9724
Length of users dict: 488, Length of users history lens: 488


In [8]:
eval_users_num = int(users_num * 0.2)
eval_movies_num = movies_num
print(f'Evaluation set: \n# users: {eval_users_num}, # movies: {eval_movies_num}')

eval_users_dict = {k: users_dict[k] for k in range(users_num-eval_users_num, users_num)}
eval_users_history_lens = users_history_lens[-eval_users_num:]
print(f'Length of users dict: {len(eval_users_dict)}, Length of users history lens: {len(eval_users_history_lens)}')

Evaluation set: 
# users: 122, # movies: 9724
Length of users dict: 122, Length of users history lens: 122


# Evaluation
Variables:
* movies_id_to_movies={movieId: [title, genres],}
* users_dict={userId:[movieId, rating],}
* users_history_lens=[length,]
* users_num=611
* movies_num=193610

In [9]:
import tensorflow as tf
import itertools
import time

from envs import OfflineEnv
from ddpg import DDPGAgent

2025-04-26 17:20:08.500971: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
def calculate_ndcg(rel, irel):
    dcg = 0
    idcg = 0
    rel = [1 if r>0 else 0 for r in rel]
    for i, (r, ir) in enumerate(zip(rel, irel)):
        dcg += (r)/np.log2(i+2)
        idcg += (ir)/np.log2(i+2)
    return dcg, idcg

In [28]:
def evaluate(recommender, env, check_movies=False, top_k=10):
    episode_reward = 0
    steps = 0
    mean_precision = 0
    mean_ndcg = 0
    mean_recall = 0
    mean_ap = 0

    user_id, items_ids, done = env.reset()
    if check_movies:
        print(f'user_id : {user_id}, rated_items_length:{len(env.user_items)}')
        print('history items : \n', np.array(env.get_items_names(items_ids)))

    while not done:
        user_eb = recommender.embedding_network.get_layer('user_embedding')(np.array(user_id))
        items_eb = recommender.embedding_network.get_layer('movie_embedding')(np.array(items_ids))
        
        state = recommender.srm_ave([np.expand_dims(user_eb, axis=0), np.expand_dims(items_eb, axis=0)])
        
        action = recommender.actor.network(state)
        
        recommended_item = recommender.recommend_item(action, env.recommended_items, top_k=top_k)
        if check_movies:
            print(f'recommended items ids : {recommended_item}')
            print(f'recommened items : \n {np.array(env.get_items_names(recommended_item), dtype=object)}')
        
        next_items_ids, reward, done, _ = env.step(recommended_item, top_k=top_k)
        if top_k:
            correct_list = [1 if r > 0 else 0 for r in reward]
            # NDCG calculation
            dcg, idcg = calculate_ndcg(correct_list, [1 for _ in range(len(reward))])
            mean_ndcg += dcg / idcg
            
            # Precision calculation
            correct_num = top_k - correct_list.count(0)
            mean_precision += correct_num / top_k

            # Recall calculation
            recall_k = correct_num / env.num_relevant_items if hasattr(env, 'num_relevant_items') and env.num_relevant_items > 0 else 0
            mean_recall += recall_k

            # MAP (Average Precision at K)
            num_hits = 0
            ap_sum = 0.0
            for idx, rel in enumerate(correct_list):
                if rel == 1:
                    num_hits += 1
                    ap_sum += num_hits / (idx + 1)
            average_precision = ap_sum / max(sum(correct_list), 1)  # Simplified sum
            mean_ap += average_precision
            
        reward = np.sum(reward)
        items_ids = next_items_ids
        episode_reward += reward
        steps += 1
        
        if check_movies:
            print(f'precision : {correct_num/top_k}, dcg : {dcg:0.3f}, idcg : {idcg:0.3f}, ndcg : {dcg/idcg:0.3f}, reward : {reward}')
            print()

    if check_movies:
        print(f'precision : {mean_precision/steps}, ndcg : {mean_ndcg/steps}, recall : {mean_recall/steps}, MAP : {mean_ap/steps}, episode_reward : {episode_reward}')
        print()
    
    return mean_precision / steps, mean_ndcg / steps, mean_recall / steps, mean_ap / steps

In [12]:
# get the saved model dir
saved_actor = '/Users/vivianyan/Desktop/Reinforcement-Learning/Project/RL-Movie-Rec-Sys/DDPG/save_model/trail-2025-04-25-22/actor_10_fixed.weights.h5'
saved_critic = '/Users/vivianyan/Desktop/Reinforcement-Learning/Project/RL-Movie-Rec-Sys/DDPG/save_model/trail-2025-04-25-22/critic_10_fixed.weights.h5'

In [29]:
tf.keras.backend.set_floatx('float64')

STATE_SIZE = 10
TOP_K = 10 
movies_num = max(ratings_df['movieId']) + 1

sum_precision = 0
sum_ndcg = 0
sum_recall = 0
sum_map = 0

end_evaluation = 10

for i, user_id in enumerate(eval_users_dict.keys()):
    env = OfflineEnv(eval_users_dict, eval_users_history_lens, movies_id_to_movies, STATE_SIZE, fix_user_id=user_id)
    recommender = DDPGAgent(env, users_num, movies_num, STATE_SIZE)
    recommender.actor.build_networks()
    recommender.critic.build_networks()
    recommender.load_model(saved_actor, saved_critic)

    precision, ndcg, recall, map_ = evaluate(recommender, env, check_movies=True, top_k=TOP_K)
    sum_precision += precision
    sum_ndcg += ndcg
    sum_recall += recall
    sum_map += map_

    if i+1 > end_evaluation:
        break

n_users = min(end_evaluation+1, len(eval_users_dict))
print(f'precision@{TOP_K} : {sum_precision/n_users:.4f}, ndcg@{TOP_K} : {sum_ndcg/n_users:.4f}, recall@{TOP_K} : {sum_recall/n_users:.4f}, MAP@{TOP_K} : {sum_map/n_users:.4f}')

/opt/anaconda3/envs/ddpg/lib/python3.9/site-packages/keras/src/layers/layer.py:1410: UserWarning: Layer 'drr_ave_state_representation_47' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating 

user_id : 489, rated_items_length:648
history items : 
 [["Dracula (Bram Stoker's Dracula) (1992)"
  'Fantasy|Horror|Romance|Thriller']
 ['Lady and the Tramp (1955)' 'Animation|Children|Comedy|Romance']
 ['Lethal Weapon 3 (1992)' 'Action|Comedy|Crime|Drama']
 ['Runaway Bride (1999)' 'Comedy|Romance']
 ["Boys Don't Cry (1999)" 'Drama']
 ['Tootsie (1982)' 'Comedy|Romance']
 ['Hook (1991)' 'Adventure|Comedy|Fantasy']
 ['13th Warrior, The (1999)' 'Action|Adventure|Fantasy']
 ['Englishman Who Went Up a Hill But Came Down a Mountain, The (1995)'
  'Comedy|Romance']
 ['Conan the Barbarian (1982)' 'Action|Adventure|Fantasy']]
recommended items ids : [136024    999    353  97024  26791   3422 188751   7236   1942  33677]
recommened items : 
 [['The Professional: Golgo 13 (1983)' 'Action|Animation|Crime']
 ['2 Days in the Valley (1996)' 'Crime|Film-Noir']
 ['Crow, The (1994)' 'Action|Crime|Fantasy|Thriller']
 ["Rust and Bone (De rouille et d'os) (2012)" 'Drama|Romance']
 ['Shining Through (1992)

/opt/anaconda3/envs/ddpg/lib/python3.9/site-packages/keras/src/layers/layer.py:1410: UserWarning: Layer 'drr_ave_state_representation_48' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating 

user_id : 490, rated_items_length:113
history items : 
 [['Father of the Bride Part II (1995)' 'Comedy']
 ['Mask of Zorro, The (1998)' 'Action|Comedy|Romance']
 ['Nine Months (1995)' 'Comedy|Romance']
 ["My Best Friend's Wedding (1997)" 'Comedy|Romance']
 ['(500) Days of Summer (2009)' 'Comedy|Drama|Romance']
 ['Inception (2010)' 'Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX']
 ['Stranger than Fiction (2006)' 'Comedy|Drama|Fantasy|Romance']
 ['Little Miss Sunshine (2006)' 'Adventure|Comedy|Drama']
 ['Social Network, The (2010)' 'Drama']
 ['Prestige, The (2006)' 'Drama|Mystery|Sci-Fi|Thriller']]
recommended items ids : [  2017  99191  90405  33677 134334   2782  65982   2987  32898  54286]
recommened items : 
 [['Babes in Toyland (1961)' 'Children|Fantasy|Musical']
 ['Campfire Tales (1997)' 'Horror']
 ['In Time (2011)' 'Crime|Sci-Fi|Thriller']
 ['Rock School (2005)' 'Documentary']
 ['Phir Hera Pheri (2006)' 'Comedy']
 ['Pit and the Pendulum (1961)' 'Horror']
 ['Outlander (2008)' 'Act

/opt/anaconda3/envs/ddpg/lib/python3.9/site-packages/keras/src/layers/layer.py:1410: UserWarning: Layer 'drr_ave_state_representation_49' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating 

...

user_id : 497, rated_items_length:51
history items : 
 [['Shawshank Redemption, The (1994)' 'Crime|Drama']
 ['Life Is Beautiful (La Vita Ã¨ bella) (1997)'
  'Comedy|Drama|Romance|War']
 ['Saving Private Ryan (1998)' 'Action|Drama|War']
 ['Matrix, The (1999)' 'Action|Sci-Fi|Thriller']
 ['Good Will Hunting (1997)' 'Drama|Romance']
 ['Sixth Sense, The (1999)' 'Drama|Horror|Mystery']
 ['Silence of the Lambs, The (1991)' 'Crime|Horror|Thriller']
 ['Batman vs. Robin (2015)' 'Action|Adventure|Animation']
 ['Jumanji (1995)' 'Adventure|Children|Fantasy']
 ['Ace Ventura: When Nature Calls (1995)' 'Comedy']]
recommended items ids : [ 84772  75341   1566   2782   3739 136564   4002  99191  54286  32898]
recommened items : 
 [['Paul (2011)' 'Adventure|Comedy|Sci-Fi']
 ['Remember Me (2010)' 'Drama|Romance']
 ['Hercules (1997)' 'Adventure|Animation|Children|Comedy|Musical']
 ['Pit and the Pendulum (1961)' 'Horror']
 ['Trouble in Paradise (1932)' 'Comedy|Romance']
 ['Macbeth (2015)' 'Drama']
 ['Planes

/opt/anaconda3/envs/ddpg/lib/python3.9/site-packages/keras/src/layers/layer.py:1410: UserWarning: Layer 'drr_ave_state_representation_56' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating 

user_id : 498, rated_items_length:35
history items : 
 [['Batman Forever (1995)' 'Action|Adventure|Comedy|Crime']
 ['Cliffhanger (1993)' 'Action|Adventure|Thriller']
 ['Pulp Fiction (1994)' 'Comedy|Crime|Drama|Thriller']
 ['Apollo 13 (1995)' 'Adventure|Drama|IMAX']
 ['Batman (1989)' 'Action|Crime|Thriller']
 ['Dances with Wolves (1990)' 'Adventure|Drama|Western']
 ['True Lies (1994)' 'Action|Adventure|Comedy|Romance|Thriller']
 ['Clear and Present Danger (1994)' 'Action|Crime|Drama|Thriller']
 ['Aladdin (1992)' 'Adventure|Animation|Children|Comedy|Musical']
 ['Dumb & Dumber (Dumb and Dumber) (1994)' 'Adventure|Comedy']]
recommended items ids : [ 75341   1566 134004   3739 184791  99191 136564   4002  54286  32898]
recommened items : 
 [['Remember Me (2010)' 'Drama|Romance']
 ['Hercules (1997)' 'Adventure|Animation|Children|Comedy|Musical']
 ['What Love Is (2007)' 'Comedy|Romance']
 ['Trouble in Paradise (1932)' 'Comedy|Romance']
 ['Fred Armisen: Standup for Drummers (2018)' 'Comedy']
 

/opt/anaconda3/envs/ddpg/lib/python3.9/site-packages/keras/src/layers/layer.py:1410: UserWarning: Layer 'drr_ave_state_representation_57' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating 

user_id : 499, rated_items_length:27
history items : 
 [['Tank Girl (1995)' 'Action|Comedy|Sci-Fi']
 ['Lost Boys, The (1987)' 'Comedy|Horror|Thriller']
 ['Muppet Christmas Carol, The (1992)' 'Children|Comedy|Musical']
 ['Pale Rider (1985)' 'Western']
 ['Hedwig and the Angry Inch (2000)' 'Comedy|Drama|Musical']
 ['Plan 9 from Outer Space (1959)' 'Horror|Sci-Fi']
 ['Man Who Knew Too Much, The (1956)' 'Adventure|Drama|Mystery|Thriller']
 ['Dracula (1931)' 'Horror']
 ['Death Proof (2007)' 'Action|Adventure|Crime|Horror|Thriller']
 ['Velvet Goldmine (1998)' 'Drama']]
recommended items ids : [134004   2782   1566 184791   3739 136564  99191   4002  54286  32898]
recommened items : 
 [['What Love Is (2007)' 'Comedy|Romance']
 ['Pit and the Pendulum (1961)' 'Horror']
 ['Hercules (1997)' 'Adventure|Animation|Children|Comedy|Musical']
 ['Fred Armisen: Standup for Drummers (2018)' 'Comedy']
 ['Trouble in Paradise (1932)' 'Comedy|Romance']
 ['Macbeth (2015)' 'Drama']
 ['Campfire Tales (1997)' 'Hor